Используя данные о покупках, выясните, какие пары товаров пользователи чаще всего покупают вместе. Необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.

Результат свести в таблицу со следующими данными:
* 1_Товар – наименование первого товара
* 2_Товар – наименование второго товара
* Встречаемость – число раз, когда такая пара была встречена

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = 'https://stepik.org/media/attachments/lesson/409319/test1_completed.csv'

Считываем данные:

In [3]:
vegan = pd.read_csv(path)

In [4]:
vegan.head(5)

,id,Товар,Количество
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0
3,17119,Лук репчатый,2.5
4,17119,Малина свежая,1.0


Посмотрим размер датафрейма, типы данных:

In [5]:
vegan.shape

(43514, 3)

In [6]:
vegan.dtypes

id              int64
Товар          object
Количество    float64
dtype: object

Удалим данные с количеством купленных товаров `Количество`. Для исследования паттернов покупок нам они не нужны.

In [7]:
vegan.drop(columns = 'Количество', inplace = True)

Переименуем колонки для удобства обращения к данным:

In [8]:
vegan = vegan.rename(columns = {'id':'purchase_id', 'Товар':'product_name'})
vegan.head()

,purchase_id,product_name
0,17119,Лимон
1,17119,Лимон оранжевый
2,17119,Лук-порей
3,17119,Лук репчатый
4,17119,Малина свежая


Количество покупок:

In [9]:
vegan.purchase_id.nunique()

3273

Количество наименований товаров:

In [10]:
vegan.product_name.nunique()

199

Посмотрим на самые часто встречающиеся товары (пока не пары). Для контроля результата.

In [11]:
vegan.product_name.value_counts().sort_values(ascending = False).head(8)

Огурцы Луховицкие          1022
Арбуз                       978
Укроп                       828
Бананы                      691
Кабачки                     662
Лук репчатый                606
Персики азербайджанские     596
Баклажаны грунтовые         590
Name: product_name, dtype: int64

Проверка на пропущенные данные:

In [12]:
vegan.isna().sum()

purchase_id     0
product_name    0
dtype: int64

Для решения задачи создадим датафрейм со всеми парами товаров. 

Функция `bascet_couple` возвращает датафрейм с парами товаров из асортиментного ряда, который есть в исходных данных.

In [13]:
def bascet_couple(product_list):
    """
    функция формирует из ассортимента товара в покупке серию с парами товаров
    два цикла: в первом фиксируем 1-ое наименование товара (level_1), 
    второй - проходит по всем оставшимся товарам (level_2),
    таким образом создаются все комбинации пар товаров,
    далее смещаемся на следующее наименование товара и проходим цикл  
    """
    product_list = sorted(product_list)
    list_of_pairs = [(product_list[level_1], product_list[level_2]) 
                     for level_1 in range(len(product_list)) 
                     for level_2 in range(level_1+1,len(product_list))]
    return pd.Series(list_of_pairs, dtype = 'str')

Запускаем функцию `bascet_couple` для создания датафрейма с парами продуктов из покупок и считаем количество повторов пар товаров, переименовываем  названия колонок и делаем сортировку по `Встречаемости`:

In [14]:
%%time
vegan_pair = vegan.groupby('purchase_id')\
                  .product_name\
                  .apply(bascet_couple)\
                  .value_counts()\
                  .reset_index()\
                  .rename(columns = {'index':'pair_products', 'product_name':'Встречаемость'})\
                  .sort_values('Встречаемость', ascending = False)


CPU times: user 11.3 s, sys: 127 ms, total: 11.5 s
Wall time: 11.4 s


Посмотрим на первые строки получившегося датафрейма с паттернами покупок:

In [15]:
vegan_pair.head()

,pair_products,Встречаемость
0,"(Огурцы Луховицкие, Укроп)",431
1,"(Петрушка, Укроп)",408
2,"(Арбуз, Огурцы Луховицкие)",345
3,"(Кабачки, Огурцы Луховицкие)",326
4,"(Кинза, Укроп)",303


Дорабатываем получившийся датафрейм с паттернами покупок в соответствии с заданием:
* разобъем колонку с парами продуктов на колонки требуемые в задаче `1_Товар`,`2_Товар`
* удалим колонку с парами продуктов `pair_products`
* изменим очередность колонок в результирующем датафрейме в соответстви с заданием
* выведем первые 5 строк итогового датафрейма с наиболее распространенными паттернами покупок

In [16]:
vegan_pair['1_Товар'] = vegan_pair.pair_products.str[0]
vegan_pair['2_Товар'] = vegan_pair.pair_products.str[1]
vegan_pair.drop(columns = 'pair_products', inplace = True)
columns_result = ['1_Товар', '2_Товар', 'Встречаемость']
vegan_pair = vegan_pair.reindex(columns = columns_result)
vegan_pair.head()

,1_Товар,2_Товар,Встречаемость
0,Огурцы Луховицкие,Укроп,431
1,Петрушка,Укроп,408
2,Арбуз,Огурцы Луховицкие,345
3,Кабачки,Огурцы Луховицкие,326
4,Кинза,Укроп,303


Вывод таблицы с 5 наиболее встречающимися паттернов покупок:

In [17]:
top_5 = (vegan_pair.iloc[:5]
        .style
        .hide_index()
        .set_caption('Топ-5 паттернов продуктов'))
top_5

1_Товар,2_Товар,Встречаемость
Огурцы Луховицкие,Укроп,431
Петрушка,Укроп,408
Арбуз,Огурцы Луховицкие,345
Кабачки,Огурцы Луховицкие,326
Кинза,Укроп,303
